In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import molsysmt as msm

# Select

Elements selections is probably the most frequently task when we work with molecular systems. There are many circumstances under which we need to know list of elements acomplishing a certain condition. We probably need, for instance, to calculate de contact map between CA atoms from two chains, or to remove the solvent atoms or to know how many 'HIS' residues there are in a peptide. All these conditions can be expresed as a sentence that the query over elements needs to match. Each library or MD engine or molecular visualization software have each own syntaxis to write this sentence. You can see different examples in MDTraj, PyTraj, Amber, Pymol or VMD.

## MolSysMT selection syntaxis

MolSysMT has its own selection syntaxis based on the attributes of the elements as atoms, groups, molecules, etc. Lets load a molecular system to explain the logic behind this syntaxis:

In [3]:
molecular_system = msm.convert('pdb_id:1tcd', to_form='molsysmt.MolSys')

/home/diego/projects/MolSysMT/molsysmt/item/mmtf_MMTFDecoder/to_molsysmt_Topology.py:34: UserWarning: The structure in the PDB has biological assemblies. There are geometrical transformations proposed in the structure. See the following issue in the source code repository: https://github.com/uibcdf/MolSysMT/issues/33
  warnings.warn(warning_message)


A molecular system encoded as the native form 'MolSys' has a pandas DataFrame with the atoms breakdown:

In [ ]:
molecular_system.topology.atoms_dataframe

As you can see, the column names are the fundamental attributes of the molecular system elements:

In [ ]:
print(molecular_system.topology.atoms_dataframe.columns)

The syntaxis proposed by Pandas to perform queries in a pandas.DataFrame is the base of the MolSysMT selection procedure. The boolean syntaxis of Pandas includes the following words and symbols:

<center>

| Word | Symbol | Meaning |
|---|---|---|
| and | & | and |
| or | \| | or |
| not | ~ | not |
| in | | in |
|  | == | equal |
|  | != | not equal |
|  | < | less than |
|  | <= | less or equal than |
|  | > | greater than |
|  | >= | greater or equal than |

</center>

As such, the selection sentence can also include the reference to external lists. Lets see some simple examples.

## element selections with atom indices

In [ ]:
msm.select(molecular_system, selection=[0,1,2])

In [ ]:
msm.select(molecular_system, element='group', selection=[0,1,2,3,4,5,6,7,8,9,10,11])

In [ ]:
msm.select(molecular_system, element='molecule', selection=[3900, 3910, 3920])

## Simple atoms selection by their attributes or properties
The following are some examples where a list of atoms is obtained matching some selection criteria:

In [ ]:
# Atoms with name C
msm.select(molecular_system, 'atom_name == "C"')

In [ ]:
# Atoms with name CA or CB
msm.select(molecular_system, 'atom_name in ["CA","CB"]')

In [ ]:
# Atoms of type C or N
msm.select(molecular_system, 'atom_type==["C","N"]')

In [ ]:
# Heavy atoms
msm.select(molecular_system, 'not atom_type=="H"')

In [ ]:
# Atoms of type C not named CA
msm.select(molecular_system, 'atom_type=="C" and not atom_name=="CA"')

In [ ]:
# Atoms not named CA, CB or C
msm.select(molecular_system, 'atom_name!=["CA","CB","C"]')

In [ ]:
# Atoms with id number lower than 10
msm.select(molecular_system, 'atom_id<10')

In [ ]:
# Atoms with id number lower than 10 and higher or equal than 3
msm.select(molecular_system, 'atom_id<10 and atom_id>=3')

## Including other elements attributes

Atoms can be selected using attributes of other the other elements in the hierarchical organization of the molecular system: 'group', 'component', 'molecule', 'chain', 'entity' or 'bioassembly'. You can find further information of these elements in XXX. These are some examples of selection sentences including other criteria than atoms attributes:

In [ ]:
# Atoms belonging to molecules of type water.
msm.select(molecular_system, 'molecule_type=="water"')

In [ ]:
# Heavy atoms belonging to molecules of type protein.
msm.select(molecular_system, 'molecule_type=="protein" and atom_type!="H" and group_index==3')

In [ ]:
# Atoms belonging to residues named GLY, ALA or VAL in chain id A.
msm.select(molecular_system, 'group_name==["GLY","ALA","VAL"] and chain_id=="A"') 

## Including external variables

Pandas query method allows the use of external variables in the logical sentence. To include them, variables names have to be preceded by the character '@'. Lets illustrate its use with some examples:

In [46]:
# Atoms in groups with indices 10, 11 or 12.
indices=[10,11,12]
msm.select(molecular_system, 'group_index==@indices')

array([77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93,
       94, 95, 96, 97, 98, 99])

In [45]:
# Atoms named CA, C, O or N in groups with indices 10 to 29.
indices=list(range(10,30))
atoms=["CA", "C", "O", "N"]
msm.select(molecular_system, 'atom_name==@atoms & atom_index==@indices') 

array([10, 11, 12, 16, 17, 18, 19, 25, 26, 27, 28])

## Including mask filters

Although including masks is not really necessary, `molsysmt.select()` has an optional input argument to do so:

In [ ]:
# Atoms named C with atom index in range 10 to 29
indices=list(range(10,30))
msm.select(molecular_system, 'atom_name=="C"', mask=indices)

The use of masks can always be avoid using the logical sentence:

In [ ]:
# Atoms named C with atom index in range 10 to 29
indics=list(range(10,30))
msm.select(molecular_system, 'atom_name=="C" and atom_index in @indics')

## Selection of other elements

The selection method of MolSysMT can also return other elements indices than atoms. As many methods in this library, `molsysmt.select()` has an input argument named `element` to select the elements nature of the output list of indices. Lets see some examples:

In [44]:
# Groups with indices equal to 0, 100 or 200
indices=[0,100,200]
msm.select(molecular_system, 'group_index==@indices', element='group')

array([  0, 100, 200])

In [ ]:
# Groups with name "ALA"
msm.select(molecular_system, 'group_name=="ALA"', element='group')

In [ ]:
# Groups of atoms index 34, 44 or 64
msm.select(molecular_system, 'atom_index==[34,44,64]', element='group')

In [ ]:
# Groups belonging to chain id A or C and molecule of type anything but water
msm.select(molecular_system, 'chain_id==["A","C"] and molecule_type!="water"', element='group')

In [ ]:
# Groups of molecules of type water
msm.select(molecular_system, 'molecule_type=="water"', element='group')

In [ ]:
# Molecules of type water
msm.select(molecular_system, 'molecule_type=="water"', element='molecule')

In [ ]:
# Chains with molecules of type water
msm.select(molecular_system, 'molecule_type=="water"', element='chain')

In [ ]:
# Bonds in group index 5
msm.select(molecular_system, 'group_index==5', element='bond')

Finnally, notice that `mask` is always acting over the elemented elements:

In [ ]:
# Atoms with index from 0 to 4 and from 0 to 2
msm.select(molecular_system, 'atom_index in [0,1,2,3,4]', mask=[0,1,2], element='atom')

In [ ]:
# Groups with index from 0 to 4 and from 0 to 2
msm.select(molecular_system, 'group_index in [0,1,2,3,4]', mask=[0,1,2], element='group')

## Special selection tools

A selection of elements within a certain distance of a set of elements can be obtained using the string `within ... of`:

In [10]:
msm.select(molecular_system, 'chain_id=="A" within 0.3 nm of chain_id=="B"')

array([ 89, 480, 527, 547, 550, 552, 554, 566, 723, 734])

In [11]:
msm.select(molecular_system, 'chain_id=="A" not within 7.8 nanometers of chain_id=="B"')

array([1521, 1522, 1723, 1724])

In [12]:
msm.select(molecular_system, 'chain_id=="A" within 0.3 nm without pbc of chain_id=="B"')

array([ 89, 480, 527, 547, 550, 552, 554, 566, 723, 734])

In [13]:
msm.select(molecular_system, 'chain_id=="A" within 0.3 nm with pbc of chain_id=="B"')

array([ 89, 480, 527, 547, 550, 552, 554, 566, 723, 734])

In [14]:
msm.select(molecular_system, '(atom_name=="N" and chain_id=="A") within 3 angstroms of (atom_type=="O" and molecule_type=="water")')

array([ 119,  213,  473,  531,  654,  696,  799, 1049])

In [15]:
msm.select(molecular_system, '(atom_name=="CA" and chain_id=="A") within 0.5 nm of (atom_name=="CA" and chain_id=="B")',
          element='group')

array([10, 42, 62, 72, 73])

Atoms bonded to specific atoms can also be selected with `bonded to`:

In [43]:
msm.select(molecular_system, 'atom_name=="N" bonded to atom_type=="C"')

array([   0,    9,   16,   25,   32,   40,   45,   50,   55,   63,   77,
         86,   92,  100,  104,  110,  119,  125,  133,  141,  148,  155,
        163,  171,  180,  187,  195,  203,  208,  213,  220,  231,  239,
        249,  257,  264,  273,  279,  286,  293,  298,  305,  312,  323,
        331,  341,  349,  356,  364,  371,  380,  385,  396,  404,  411,
        419,  426,  435,  446,  455,  463,  468,  473,  482,  490,  495,
        503,  510,  521,  527,  531,  536,  547,  554,  558,  567,  574,
        580,  588,  597,  605,  613,  622,  630,  642,  646,  654,  660,
        674,  681,  688,  696,  700,  710,  716,  725,  736,  747,  755,
        767,  779,  783,  792,  799,  807,  816,  824,  831,  836,  845,
        854,  861,  866,  875,  880,  886,  891,  896,  900,  911,  921,
        928,  936,  943,  949,  956,  960,  969,  976,  984,  993, 1002,
       1013, 1022, 1027, 1031, 1042, 1049, 1054, 1059, 1066, 1073, 1081,
       1088, 1097, 1105, 1110, 1115, 1122, 1127, 11

In [42]:
msm.select(molecular_system, '(all not bonded to atom_type==["H","N","C","O"]) and molecule_type=="protein"')

array([ 363, 1714, 2275, 3626])

In [41]:
msm.select(molecular_system, '(atom_type=="O" and chain_id=="A") bonded to (atom_type=="C" and chain_id=="A")')

array([   3,   12,   19,   23,   28,   35,   43,   48,   53,   58,   61,
         66,   80,   89,   95,   98,  103,  107,  109,  113,  117,  118,
        122,  124,  128,  136,  144,  151,  158,  166,  174,  178,  179,
        183,  185,  190,  198,  201,  206,  211,  216,  218,  223,  234,
        237,  238,  242,  252,  255,  256,  260,  267,  271,  276,  282,
        289,  296,  301,  308,  310,  315,  326,  334,  344,  352,  359,
        367,  369,  374,  383,  388,  399,  407,  409,  414,  417,  422,
        429,  438,  449,  453,  458,  466,  471,  476,  480,  485,  488,
        493,  498,  506,  508,  513,  524,  526,  530,  534,  539,  550,
        552,  557,  561,  565,  566,  570,  577,  579,  583,  591,  595,
        600,  608,  616,  625,  628,  629,  633,  641,  645,  649,  657,
        659,  663,  677,  684,  691,  699,  703,  713,  715,  719,  723,
        724,  728,  739,  750,  758,  766,  770,  778,  782,  786,  790,
        791,  795,  797,  802,  805,  810,  814,  8

And both, `within .. of` and `bonded to`, can be mixed in the same selection sentence:

In [40]:
msm.select(molecular_system, '((atom_name=="N" and chain_id=="A") bonded to atom_type=="C") within 3 angstroms of (atom_type=="O" and molecule_type=="water")')

array([ 119,  213,  473,  531,  654,  696,  799, 1049])

## Syntaxis translation

MolSysMT is prepared to easily interact with other tools. The main goal of this library is providing with a set of pipes and joins to set up your workflows, keeping simple the integration of other tools. But different tools have different selection syntaxis. Learning how to use the selection syntaxis of MDTraj, ParmEd or NGLview is something very useful. Those are tools that we all use frequently in our labs. But it happens that we forget soon the rules of each tool. To keep a unique selection syntaxis in your projects, MolSysMT includes the input argument `to_syntaxis` in the method `molsysmt.select()`. Lets illustrate some examples:

In [ ]:
msm.select(molecular_system, selection='group_index==[3,4,5]', to_syntaxis='NGLView')

In [ ]:
msm.select(molecular_system, selection='group_index==[3,4,5]', to_syntaxis='MDTraj')

The output string can be obtained, if the selection is done over other elementted elements, as a sequence of groups or chains:

In [ ]:
msm.select(molecular_system, element='group', selection='group_index==[3,4,5]', to_syntaxis='NGLView')

In [ ]:
msm.select(molecular_system, element='group', selection='group_index==[3,4,5]', to_syntaxis='MDTraj')

### Output syntaxis supported

MolSysMT translates selection sentences from its own native syntaxis to NGLview, MDTraj, Pytraj, ParmEd and AMBER.

## Using your favourite selection syntaxis

Already implemented: testing and documenting need it.